In [0]:
!pip install -q tensorflow==2.1.0

In [0]:
import tensorflow as tf

In [0]:
print(tf.__version__)

2.1.0


In [0]:
from tensorflow.keras import datasets, layers, models, regularizers, optimizers
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import numpy as np

In [0]:
EPOCHS = 50
NUM_CLASSES = 10

In [0]:
def load_data():
  (X_train, y_train), (X_test, y_test) = datasets.cifar10.load_data()
  X_train = X_train.astype('float32')
  X_test = X_test.astype('float32')

  #normalize
  mean = np.mean(X_train, axis=(0, 1, 2, 3))
  std = np.std(X_train, axis=(0, 1, 2, 3))
  X_train = (X_train-mean) / (std + 1e-7)
  X_test = (X_test-mean)/ (std + 1e-7)

  return X_train, y_train, X_test, y_test

In [0]:
def build_model():
  model =  models.Sequential()

  #1st block
  model.add(layers.Convolution2D(32, (3, 3), padding='same', input_shape=X_train.shape[1:], activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(32, (3, 3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.2))

  #2nd block
  model.add(layers.Convolution2D(64, (3, 3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(64, (3, 3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.3))

  #3rd block
  model.add(layers.Convolution2D(128, (3, 3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.Conv2D(128, (3, 3), padding='same', activation='relu'))
  model.add(layers.BatchNormalization())
  model.add(layers.MaxPooling2D(pool_size=(2, 2)))
  model.add(layers.Dropout(0.4))

  # Dense
  model.add(layers.Flatten())
  model.add(layers.Dense(NUM_CLASSES, activation='softmax'))

  return model

  model.summary()


In [0]:
(X_train, y_train, X_test, y_test) = load_data()

In [0]:
model = build_model()

In [0]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='RMSprop', metrics=['accuracy'])

In [0]:
import datetime
from tensorflow.keras.callbacks import TensorBoard

In [0]:
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y%m%d - %H%M%S')
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [0]:
batch_size = 64
model.fit(X_train, y_train, batch_size=batch_size, epochs=EPOCHS, validation_data=(X_test, y_test))

Train on 50000 samples, validate on 10000 samples
Epoch 1/50
50000/50000 [==============================] - 422s 8ms/sample - loss: 1.7789 - accuracy: 0.4601 - val_loss: 1.2147 - val_accuracy: 0.6127
Epoch 2/50
50000/50000 [==============================] - 421s 8ms/sample - loss: 1.0668 - accuracy: 0.6445 - val_loss: 0.8966 - val_accuracy: 0.6916
Epoch 3/50
50000/50000 [==============================] - 420s 8ms/sample - loss: 0.8424 - accuracy: 0.7151 - val_loss: 0.7599 - val_accuracy: 0.7397
Epoch 4/50
50000/50000 [==============================] - 418s 8ms/sample - loss: 0.7240 - accuracy: 0.7537 - val_loss: 0.7141 - val_accuracy: 0.7611
Epoch 5/50
50000/50000 [==============================] - 417s 8ms/sample - loss: 0.6390 - accuracy: 0.7780 - val_loss: 0.6497 - val_accuracy: 0.7844
Epoch 6/50
50000/50000 [==============================] - 419s 8ms/sample - loss: 0.5837 - accuracy: 0.7988 - val_loss: 0.6358 - val_accuracy: 0.7807
Epoch 7/50
50000/50000 [==========================

In [0]:
score = model.evaluate(X_test, y_test, batch_size=batch_size)

10000/10000 [==============================] - 19s 2ms/sample - loss: 0.4936 - accuracy: 0.8631


In [0]:
print('Test Score: ', score[0])
print('Test Accuracy: ', score[1])

Test Score:  0.49358755321502684
Test Accuracy:  0.8631


In [0]:
model.save('CNNDeep.h5')